In [36]:
tf_agents_image = 'gcr.io/stobias-dev/blob-gym:latest'
model_name = "blob_bot"
storage_class='basic'

In [90]:
import json
from string import Template

#import kfp
from kfp import components
from kfp.components import func_to_container_op
import kfp.dsl as dsl
import kfp.components as comp
from kubernetes import client as k8s_client

In [111]:
def create_agent_data_vol(model_name,storage_class):
    return dsl.VolumeOp(
        name=str(model_name),
        resource_name=str(model_name),
        size='10Gi',
        modes=['ReadWriteMany'],
        storage_class=storage_class
    )

def run_tf_agent(vol,train_steps):
    """Creates a training run (Creates a data vol along the way.)"""
    return dsl.ContainerOp(
        name='run_agent',
        image=tf_agents_image,
        command=['python',
                 '/app/train.py',
                '-root_dir=/data/gym/blob2d-v1/',
                f"--num_iterations={train_steps}",
                '--alsologtostderr'],
        pvolumes={"/data": dsl.PipelineVolume(pvc="tf-executor-build-pvc")},
        file_outputs={'output': '/tmp/mlpipeline-ui-metadata.json'}
    )

In [112]:
def blob2d_pipeline(name=model_name,storage_class=storage_class,steps=1000):
    agent = run_tf_agent("blob-env",steps)

In [113]:
#import kfp
kfp_endpoint=None

# Submit a pipeline run
from kfp_tekton import TektonClient
#blob_experiment = TektonClient(host=None).create_experiment(name='blob_experiment', 
#                                                   description='Pipeline for reinforcement learning')

TektonClient(host=None).create_run_from_pipeline_func(blob2d_pipeline,
                                                      experiment_name='blob_experiment',
                                                      arguments={'steps': 10})

RunPipelineResult(run_id=11c15776-bbe0-4f0b-901d-b1567cd60b2c)

In [ ]:
#python train.py --root_dir=$PWD/tmp/dqn/gym/blob2d-v1/ --alsologtostderr